# Notebook 03: AG5 Metrics
Calcula métricas KEC (entropia + curvatura + coerência).

In [ ]:
"""Run-All cell"""
import random, numpy as np
random.seed(42)
np.random.seed(42)

In [ ]:
from pathlib import Path
import pandas as pd
import networkx as nx
import numpy as np
import math
import yaml

In [ ]:
processed_swow = Path("../data/processed/swow/en/swow_en_tidy.csv")
processed_swow.parent.mkdir(parents=True, exist_ok=True)
df = pd.read_csv(processed_swow)
G = nx.from_pandas_edgelist(df, "cue", "response", ["weight"], create_using=nx.DiGraph())

In [ ]:
def transition_entropy(G):
    ent = {}
    for n in G.nodes:
        weights = np.array([d["weight"] for _,_,d in G.out_edges(n, data=True)])
        if weights.size == 0:
            ent[n]=0.0
            continue
        p = weights / weights.sum()
        ent[n] = float(-(p * np.log(p)).sum())
    return ent

In [ ]:
def forman_curvature(G):
    curv = {}
    for u,v,data in G.edges(data=True):
        curv[u] = curv.get(u,0)+1
        curv[v] = curv.get(v,0)+1
    return curv

In [ ]:
def community_coherence(G):
    from networkx.algorithms import community
    comms = community.greedy_modularity_communities(G.to_undirected())
    mapping = {}
    for i,c in enumerate(comms):
        for n in c:
            mapping[n]=i
    return mapping

In [ ]:
entropy = transition_entropy(G)
curv = forman_curvature(G)
comm = community_coherence(G)
metrics = []
for n in G.nodes:
    metrics.append({"node": n, "entropy": entropy.get(n,0.0), "curvature": curv.get(n,0.0), "community": comm.get(n, -1), "kec": entropy.get(n,0.0)+curv.get(n,0.0)})
metrics_df = pd.DataFrame(metrics)
out_dir = Path("../data/processed/kec")
out_dir.mkdir(parents=True, exist_ok=True)
metrics_df.to_csv(out_dir/"metrics_en.csv", index=False)